### <font color='#ff6600'>General Imports</font>  
***

In [ ]:
import csv
import time
import string
import torch
import networkx as nx
import numpy as np
from random import randint
from urllib.request import urlopen
import random
import re
import math
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 

from termcolor import cprint

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install gensim

### <font color='#ff6600'>Model Imports</font>  
***

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier 
from sklearn.model_selection import GridSearchCV 

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import RobustScaler

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

### <font color='#ff3399'>Text Pre - Processing Functions</font>  
***

In [ ]:
def preprocess_abstracts(abstract):
    # Skip last character (newline)
    abstract = abstract[:-1]

    # Convert to Lower Case
    abstract = abstract.lower()

    # Remove non-word characters
    abstract = re.sub(r'\W', ' ', abstract)

    # Remove underscores 
    abstract = re.sub(r'[-_]', ' ', abstract)

    # Remove numbers
    abstract = re.sub(r'[0-9]', ' ', abstract)
    
    # Remove all single characters
    abstract = re.sub(r'\s+[a-zA-Z]\s+', ' ', abstract)

    # Substituting multiple spaces with single space
    abstract = re.sub(r'\s+', ' ', abstract, flags=re.I)

    # Lemmatization
    abstract = abstract.split()
    abstract = [lemmatizer.lemmatize(word) for word in abstract]

    # Remove Stop Words
    abstract = [w for w in abstract if not w in stop_words]

    abstract = ' '.join(abstract)

    return abstract

In [ ]:
def preprocess_authors(author_names):
    # Skip last character (newline)
    author_names = author_names[:-1]

    # Split on comma
    possible_authors = author_names.split(',')

    # Remove non-word characters
    possible_authors = [re.sub(r'\W', ' ', f) for f in possible_authors]

    # Substituting multiple spaces with single space
    possible_authors = [re.sub(r'\s+', ' ', f, flags=re.I) for f in possible_authors]
   
    # Keep first letter of first name + whole last name
    return_authors = []
    for author in possible_authors:
        author = author.split(' ')
        author = [f for f in author if f != '' and f != ' ' and f != '  ']
        # Case 1: Surname Only
        if len(author) == 1:
            author = author[0]
        elif len(author) > 1:
            first_name_chars = ''.join([f[0] for f in author[0:-1]])
            author = first_name_chars + ' ' + author[-1]
        else:
            raise ValueError("Zero Length Name!")
        return_authors.append(author.lower())
    return return_authors

### <font color='#751aff'>Get Edgelist</font> 
***

In [ ]:
# Create a graph
G = nx.read_edgelist(urlopen('http://www.db-net.aueb.gr/nikolentzos/data_challenge_2021/edgelist.txt'), delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


### <font color='#751aff'>Get Abstracts</font>
***

In [ ]:
# Read the abstract of each paper
abstracts = dict()
corpus = []
data = urlopen('http://www.db-net.aueb.gr/nikolentzos/data_challenge_2021/abstracts.txt')
for line in data:
    decoded_line = line.decode("utf-8")
    node, abstract = decoded_line.split('|--|')
    preprocessed_abstract = preprocess_abstracts(abstract)
    corpus.append(preprocessed_abstract)
    abstracts[int(node)] = preprocessed_abstract.split(' ')

##### <font color='#b380ff'>TF-IDF Feature Creation</font> 
***

In [ ]:
# Create TF-IDF Vectors (cosine similarity solution)
vectorizer =  TfidfVectorizer(min_df=0.0, max_df=0.5,
                              sublinear_tf=True, ngram_range=(1,2))

In [ ]:
abstract_corpus = vectorizer.fit_transform(corpus)

##### <font color='#b380ff'>SVD as Topic Modeling on TF-IDF vectors</font> 
***

In [ ]:
dechonker = TruncatedSVD(n_components=200, random_state=1995)

In [ ]:
dechonker.fit(abstract_corpus)

TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
             random_state=1995, tol=0.0)

In [ ]:
abstract_corpus_svd = dechonker.transform(abstract_corpus)

In [ ]:
# Map text to set of terms (set word solution)
for node in abstracts:
    abstracts[node] = set(abstracts[node])

### <font color='#751aff'>Get Authors</font>
***

In [ ]:
# Read the abstract of each paper
authors = dict()
data = urlopen('http://www.db-net.aueb.gr/nikolentzos/data_challenge_2021/authors.txt')
for line in data:
    decoded_line = line.decode("utf-8")
    node, author = decoded_line.split('|--|')
    authors[int(node)] = preprocess_authors(author_names=author)

In [ ]:
# Map text to set of terms
for node in authors:
    authors[node] = set(authors[node])

### <font color='#006699'>Feature Generation Functions</font>
***

####   🔥  <font color='#006699'>General Functions</font>
***

In [ ]:
def fast_cosine_similarity(vec_a, vec_b, use_norm=False):
    if use_norm:
        norm_vec_a = np.linalg.norm(vec_a, ord=2)
        norm_vec_b = np.linalg.norm(vec_b, ord=2)
        if norm_vec_a > 0:
            vec_a = vec_a / norm_vec_a
        if norm_vec_b > 0:
            vec_b = vec_b / norm_vec_b

    return cosine_similarity(vec_a, vec_b)[0][0]

def get_2nd_deg(item, neighbour_set):
    tempset = set()
    for f in neighbour_set:
        tempset = tempset.union(set(G.adj[f]))
    tempset.remove(item)
    return tempset

def calc_jaccard(seta, setb):
    intersection = len(seta.intersection(setb))
    union = len(seta.union(setb))
    return intersection / (union + 1)

def calc_adar_index(G, neighbour_set):
    index = 0
    if len(neighbour_set) == 0:
        return 0
    else: 
        for neighbour in neighbour_set:
            adar = 1 / math.log(len(G.adj[neighbour]) + 1)
            index += adar
        return index

def smart_short_path(G, n1, n2):
    ### Warning Remove the edge first ###
    existed = True
    try:
        G.remove_edge(n1, n2)
    except:
        existed = False
    
    ### Now calculate the shortest exclusive path
    try:
        score = 10 / nx.shortest_path_length(G, n1, n2)
    except:
        score = 0
    
    ### Add the edge back if it existed ###
    if existed:
        G.add_edge(n1,n2)
    return score

####  😈 <font color='#006699'>Feature: Node ID</font>
***

In [ ]:
def log_id_proximity(node_1_id, node_2_id):
    node_id_distance = np.abs(node_1_id - node_2_id)
    log_node_id_distance = np.log(node_id_distance + 1e-20)
    return log_node_id_distance

#### 🚶🏻‍♀️  <font color='#006699'>DeepWalk Algorithm</font>
***



In [ ]:
class DeepWalker():
    def __init__(self, allow_dup=False, num_walks=5, walk_length=10):
        self.allow_dup = allow_dup
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.saved_walks = []
    
    def get_random_walk(self, G, node):
        walk = [node]
        for _ in range(self.walk_length-1):
            neighbors = list(G.neighbors(walk[-1]))
            target_node = random.choice(neighbors)
            if not self.allow_dup:
                retries = 0
                while target_node in walk:
                    target_node = random.choice(neighbors)
                    retries += 1
                    if retries > 5:
                        break
            walk.append(target_node)
        walk = [str(f) for f in walk]
        return walk
    

    def start_walking(self, G):
        nodes = list(G.nodes())
        for _ in range(self.num_walks):
            idx = np.random.permutation(len(nodes))
            for i in range(len(nodes)):
                node = nodes[idx[i]]
                walk = self.get_random_walk(G, node)
                self.saved_walks.append(walk)
        return

In [ ]:
def deepwalk(G, n_dim):
    print("Generating walks")
    johny = DeepWalker()
    johny.start_walking(G)
    walks = johny.saved_walks

    print("Training word2vec")
    model = Word2Vec(vector_size=n_dim, window=8, min_count=0, sg=1, workers=8)
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)

    return model

In [ ]:
n_dim = 64
model = deepwalk(G, n_dim) 

embeddings = np.zeros((n, n_dim))
for node in G.nodes():
    embeddings[node,:] = model.wv[str(node)] / (np.linalg.norm(model.wv[str(node)]))

In [ ]:
def get_deepwalk_similarity(embeddings, node_a, node_b):
    return np.dot(embeddings[node_a,:], embeddings[node_b,:])

####  🕵🏻 <font color='#006699'>Unlinked Neighbors Finder</font>
***

In [ ]:
def find_unlinked_neighbors(margin=20):
    """
        Instead of totaly random node pair, pick a close neighbour
        that does not have a link with the n1_node.
    """
    n1_ = nodes[randint(0, n-1)]
    n2_ = nodes[min(n-1,max(0,randint(n1_ - margin, n1_ + margin)))]

    while n2_ in set(G.adj[n1_]):
        margin = margin * 2
        n2_ = nodes[min(n-1,max(0,randint(n1_ - margin, n1_ + margin)))]
    return n1_, n2_

## <font color='#006699'>Text features</font>
* Number of common terms - Feature [8]
* Number of common terms normalized - Feature [9]
* Number of non-common terms - Feature [10]
* Number of non-common terms normalized - Feature [11]
* Number of common authors - Feature [12]
* Number of common authors' last names - Feature [13]
* Number of non-common authors - Feature [14]
* Number of non-common authors' last names - Feature [15]
* Cosine similarity between tfidf vectors (uni/bigram level) of abstracts - Feature [16]
* Cosine similarity between svd vectors of abstracts - Feature [17]

## <font color='#006699'>Graph features</font>

* Sum of Degrees of Nodes - Feaure [1]
* Difference of Degrees of Nodes - Feature [2]
* Sum of 2nd Degree of Nodes - Feature [3]
* Difference of 2nd Degree of Nodes - Feature [4]
* Jaccard Index 1, 2 - Features [5,6]
* Adamic Adar Index 1 - Features [7, 18]
* Shortest Path Distance - Feature [19]
* DeepWalk Similarity - Feature [20]

In [ ]:
def get_features(X_table, y_table, n1, n2, idx, y_mark=None):
    # Graph Features
    neigh_1d_0 = set(G.adj[n1])
    neigh_1d_1 = set(G.adj[n2])

    neigh_2d_0 = get_2nd_deg(n1, neigh_1d_0)
    neigh_2d_1 = get_2nd_deg(n2, neigh_1d_1)

    degree_1d_0 = len(neigh_1d_0)
    degree_1d_1 = len(neigh_1d_1)

    degree_2d_0 = len(neigh_2d_0)
    degree_2d_1 = len(neigh_2d_1)

    jaccard_1d = calc_jaccard(neigh_1d_0, neigh_1d_1)
    jaccard_2d = calc_jaccard(neigh_2d_0, neigh_2d_1)

    adar_1d = calc_adar_index(G, list(neigh_1d_0.intersection(neigh_1d_1)))
    adar_2d = calc_adar_index(G, list(neigh_2d_0.intersection(neigh_2d_1)))

    # Text Features
    abstracts_0 = abstracts[n1]
    abstracts_1 = abstracts[n2]
    authors_0 = authors[n1]
    authors_0_ln = set([f.split(' ')[-1] for f in authors_0])
    authors_1 = authors[n2]
    authors_1_ln = set([f.split(' ')[-1] for f in authors_1])

    n_terms_0 = len(abstracts_0)
    n_terms_1 = len(abstracts_1)
    n_common_terms = len(abstracts_0.intersection(abstracts_1))
    n_common_terms_norm = n_common_terms / (n_terms_0 + n_terms_1)
    n_uncommon_terms = abs(len(abstracts_0.difference(abstracts_1)))
    n_uncommon_terms_norm = n_uncommon_terms / (n_terms_0 + n_terms_1)

    n_common_authors = len(authors_0.intersection(authors_1))
    n_common_authors_last_name = len(authors_0_ln.intersection(authors_1_ln))

    n_uncommon_authors = abs(len(authors_0.difference(authors_1)))
    n_uncommon_authors_last_name = abs(len(authors_0_ln.difference(authors_1_ln)))

    pure_cos_sim = fast_cosine_similarity(abstract_corpus[n1].reshape(1,-1), abstract_corpus[n2].reshape(1,-1))
    svd_cos_sim = fast_cosine_similarity(abstract_corpus_svd[n1].reshape(1,-1), abstract_corpus_svd[n2].reshape(1,-1), use_norm=True)

    shortest_path = smart_short_path(G, n1, n2)

    ### Plug them in
    X_table[idx,0] = degree_1d_0 + degree_1d_1
    X_table[idx,1] = abs(degree_1d_0 - degree_1d_1)
    X_table[idx,2] = degree_2d_0 + degree_2d_1
    X_table[idx,3] = abs(degree_2d_0 - degree_2d_1)
    X_table[idx,4] = jaccard_1d
    X_table[idx,5] = jaccard_2d
    X_table[idx,6] = adar_1d

    X_table[idx,7] = n_common_terms
    X_table[idx,8] = n_common_terms_norm
    X_table[idx,9] = n_uncommon_terms
    X_table[idx,10] = n_uncommon_terms_norm

    X_table[idx,11] = n_common_authors
    X_table[idx,12] = n_common_authors_last_name
    X_table[idx,13] = n_uncommon_authors
    X_table[idx,14] = n_uncommon_authors_last_name

    X_table[idx, 15] = pure_cos_sim
    X_table[idx, 16] = svd_cos_sim
    X_table[idx, 17] = adar_2d
    X_table[idx, 18] = shortest_path
    X_table[idx, 19] = log_id_proximity(n1,n2)

    if y_mark is None:
        return
    else:
        y_table[idx] = y_mark
    return

## <font color='#00b38f'>Collecting Data </font>
***

In [ ]:
def negative_sampling(G:nx.Graph, start, end, validation_split:float=0.0):
    m = end - start
    X_train = np.zeros((2*m, 20))
    y_train = np.zeros(2*m)
    start_time = time.time()
    edge_list = list(G.edges())
    random.shuffle(edge_list)
    for i,edge in enumerate(edge_list):
        if i < start:
            continue
        if i < end:
            get_features(X_train, y_train, edge[0], edge[1], 2*(i-start), 1)
            n1_, n2_ = find_unlinked_neighbors(margin=20)
            get_features(X_train, y_train, n1_, n2_, 2*(i-start)+1, 0)
            if (i-start) % 5_000 == 0 and (i-start) != 0:
                checkpoint = time.time()
                cprint(f"{round(100*i/m,2)} done in {round(checkpoint - start_time,3)} seconds", 'cyan')
                np.save(f'/content/drive/MyDrive/LinkPrediction/X_train_zesti_{2*(i-start)+1}.npy', X_train[0:2*(i-start)+1,:])
                np.save(f'/content/drive/MyDrive/LinkPrediction/y_train_zesti_{2*(i-start)+1}.npy', y_train[0:2*(i-start)+1])
        else:
            break
    np.save(f'/content/drive/MyDrive/LinkPrediction/X_train_zesti_{2*(i-start)+1}.npy', X_train[0:2*(i-start)+1,:])
    np.save(f'/content/drive/MyDrive/LinkPrediction/y_train_zesti_{2*(i-start)+1}.npy', y_train[0:2*(i-start)+1])
    
    if validation_split > 0 and validation_split < 1:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, stratify=y_train, random_state=1995)
        print('Size of training matrix:', X_train.shape)
        print('Size of validation matrix:', X_val.shape)
        return X_train, y_train, X_val, y_val
    else:
        return X_train, y_train

In [ ]:
X_train_full, y_train_full = negative_sampling(G, start=0, end=50_000, validation_split=0.0)

10.0 done in 280.417 seconds
20.0 done in 537.537 seconds
30.0 done in 813.937 seconds
40.0 done in 1094.074 seconds
50.0 done in 1374.884 seconds
60.0 done in 1651.255 seconds
70.0 done in 1914.317 seconds
80.0 done in 2171.373 seconds
90.0 done in 2438.563 seconds


## <font color='#00b38f'>Training Models and evaluating on 20% Validation Split </font>
***

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=1995)

####  🦖 <font color='#b35900'>Logistic Regression </font>  

In [ ]:
clf = LogisticRegression(max_iter=10_000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
y_proba = clf.predict_proba(X_val)
logloss = log_loss(y_val, y_proba)
print(f"Assumed LogLoss: {logloss}")

              precision    recall  f1-score   support

         0.0       0.97      0.96      0.96     10000
         1.0       0.96      0.97      0.96     10000

    accuracy                           0.96     20000
   macro avg       0.96      0.96      0.96     20000
weighted avg       0.96      0.96      0.96     20000

Assumed LogLoss: 0.103292722327971


#### 🐆 <font color='#e6b800'> XGBoost 

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
best_clf_ = XGBClassifier(max_depth=5, n_estimators=250, learning_rate=0.1, 
 min_child_weight=5, gamma=0.1, subsample=0.6, colsample_bytree=0.6,
 objective= 'binary:logistic', scale_pos_rate=1, seed=1995, reg_lambda=0.01)

In [ ]:
best_clf = BaggingClassifier(base_estimator=best_clf_, n_estimators=5, random_state=1995, bootstrap=False).fit(X_train, y_train)

In [ ]:
y_pred = best_clf.predict(X_val)
print(classification_report(y_val, y_pred))
y_proba = best_clf.predict_proba(X_val)
logloss = log_loss(y_val, y_proba)
print(f"Assumed LogLoss: {logloss}")

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98     10000
         1.0       0.97      0.98      0.98     10000

    accuracy                           0.98     20000
   macro avg       0.98      0.98      0.98     20000
weighted avg       0.98      0.98      0.98     20000

Assumed LogLoss: 0.061279525939888946


#### 🦄 <font color='#33ccff'> Deep Learning - MLP 


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [ ]:
scaler = RobustScaler()
X_train_ = scaler.fit_transform(X_train)
X_val_ = scaler.transform(X_val)

In [ ]:
model = Sequential()
model.add(Input(shape=(X_train_.shape[1],)))
for i in range(0,4):
    model.add(Dense(units=200))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy(label_smoothing=0.001), metrics=['accuracy'])
callbacks = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_, to_categorical(y_train), batch_size=256, epochs=300, callbacks=callbacks, validation_data=(X_val_, to_categorical(y_val)))

Epoch 1/300
313/313 [==============================] - 3s 6ms/step - loss: 0.1234 - accuracy: 0.9542 - val_loss: 0.0809 - val_accuracy: 0.9700
Epoch 2/300
313/313 [==============================] - 2s 6ms/step - loss: 0.0933 - accuracy: 0.9664 - val_loss: 0.0761 - val_accuracy: 0.9728
Epoch 3/300
313/313 [==============================] - 2s 6ms/step - loss: 0.0898 - accuracy: 0.9672 - val_loss: 0.0756 - val_accuracy: 0.9730
Epoch 4/300
313/313 [==============================] - 2s 6ms/step - loss: 0.0866 - accuracy: 0.9687 - val_loss: 0.0731 - val_accuracy: 0.9746
Epoch 5/300
313/313 [==============================] - 2s 6ms/step - loss: 0.0845 - accuracy: 0.9688 - val_loss: 0.0716 - val_accuracy: 0.9748
Epoch 6/300
313/313 [==============================] - 2s 5ms/step - loss: 0.0822 - accuracy: 0.9706 - val_loss: 0.0710 - val_accuracy: 0.9748
Epoch 7/300
313/313 [==============================] - 2s 6ms/step - loss: 0.0822 - accuracy: 0.9704 - val_loss: 0.0701 - val_accuracy: 0.9753

In [ ]:
y_pred = np.argmax(model.predict(X_val_),axis=1)
print(classification_report(y_val, y_pred))
y_proba_3 = model.predict(X_val_)
logloss = log_loss(y_val, y_proba_3)
print(f"Assumed LogLoss: {logloss}")

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98     10000
         1.0       0.97      0.98      0.98     10000

    accuracy                           0.98     20000
   macro avg       0.98      0.98      0.98     20000
weighted avg       0.98      0.98      0.98     20000

Assumed LogLoss: 0.060630760368736755


## <font color='#00bbff'>Get Test Data and Perform Ensemble Prediction
***

In [ ]:

node_pairs = list()
data = urlopen('http://www.db-net.aueb.gr/nikolentzos/data_challenge_2021/test.txt')
for line in data:
    decoded_line = line.decode("utf-8")
    t = decoded_line.split(',')
    node_pairs.append((int(t[0]), int(t[1])))

# Create the test matrix. Use the same 15 features as above
X_test = np.zeros((len(node_pairs), 21))
for i,node_pair in enumerate(node_pairs):
    get_features(X_test, None, node_pair[0], node_pair[1], i, None)

print('Size of test matrix:', X_test.shape)

In [ ]:
best_clf = BaggingClassifier(base_estimator=best_clf_, n_estimators=5, random_state=1995, bootstrap=False).fit(X_train_full, y_train_full)
y_pred_1 = best_clf.predict_proba(X_test)[:,1]


# Write predictions to a file
predictions = zip(range(len(y_pred_1)), y_pred_1)
with open("results.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row) 